# STRMaps

You can view the map at https://s3.amazonaws.com/filestogeaux.garyscorner.net/pub/STRMap.html FOR NOW
File will remain at s3://requesterpays.garyscorner.net/datasets/html/  (requester pays use AWS CLI [--request-payer requester])

Datasets:
* https://data.nola.gov/Housing-Land-Use-and-Blight/Map-of-Short-Term-Rental-Licenses/j5u3-2ueh

In [1]:
import pandas as pd
import folium
import numpy as np
import folium.plugins

from io import StringIO

from json import JSONEncoder
encoder = JSONEncoder().encode

from datetime import datetime

In [2]:
! curl https://data.nola.gov/api/views/en36-xvxg/rows.csv?accessType=DOWNLOAD -o DataSets\Short-Term_Rental_Permit_Applications.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 4526k    0 4526k    0     0  2684k      0 --:--:--  0:00:01 --:--:-- 2686k
100 9301k    0 9301k    0     0  3751k      0 --:--:--  0:00:02 --:--:-- 3753k


In [3]:
strDf = pd.read_csv("./DataSets/Short-Term_Rental_Permit_Applications.csv")

In [4]:
#drop without location
strDf = strDf[ strDf['X'] > 0.0]

In [5]:
strDf.drop(columns=['X','Y', 'Link'],inplace=True)

In [6]:
#drop dups and withdrawn
strDf = strDf[ ~(strDf['Current Status'].isin(['Duplicate','Withdrawn'])) ]

In [7]:
strDf['Current Status'].unique()

array(['Pending', 'Revoked', 'Denied', 'Expired', 'Issued'], dtype=object)

In [8]:
#this is done in javascript now
#strDf['Link'] = strDf['Link'].map(lambda url:  f"<a target='_blank' href='{url}'>One Stop App</a>")

In [9]:
%%time

def getPoints(val):
    try:
        
        lat,lon = val[1:-1].split(', ')
        lon = float(lon)
        lat = float(lat)
        pt = (lat,lon)
        
    except Exception as e:
        print(val)
        raise(e)
        
    return pt

strDf['Location'] = strDf['Location'].map(getPoints)


Wall time: 52.9 ms


In [10]:
#now done in javascript
#strDf['Address'] = strDf.apply(lambda a: f"<a target='_blank' href='http://maps.google.com/maps?q=&layer=c&cbll={a['Location'][0]},{a['Location'][1]}'>{a['Address']}</a>",axis=1)

In [11]:
strDf['Current Status'].value_counts()

Expired    14245
Denied      7429
Pending     4005
Issued      2337
Revoked      143
Name: Current Status, dtype: int64

In [12]:
statusList = list(np.unique(strDf['Current Status']))

In [13]:
#you can uncomment that to see the map in this notebook but the notebook will balloon to 85 mb
#map

In [14]:
colMap = {
    'Guest Occupancy Limit':'Guest Limit',
    'License Holder Name':'License Holder',
    
}

strDf.rename(columns=colMap, inplace=True)

In [15]:
outputObj = {col:strDf[col].to_list() for col in strDf.columns}

In [16]:
%%time

with open('STRMapTemplate.html', 'r') as f:
    templateHTML = f.read()
    
beginHTML, endHTML = templateHTML.split('DATA_ALL')

with open('STRMap.html', 'w') as f:
    f.write(beginHTML)
    f.write(encoder(outputObj))
    f.write(endHTML)

Wall time: 203 ms


In [17]:
%%time
!aws s3 cp STRMap.html s3://requesterpays.garyscorner.net/datasets/html/

Completed 256.0 KiB/7.5 MiB (142.3 KiB/s) with 1 file(s) remainingWall time: 32.4 s

Completed 512.0 KiB/7.5 MiB (174.0 KiB/s) with 1 file(s) remaining
Completed 768.0 KiB/7.5 MiB (191.3 KiB/s) with 1 file(s) remaining
Completed 1.0 MiB/7.5 MiB (187.7 KiB/s) with 1 file(s) remaining  
Completed 1.2 MiB/7.5 MiB (180.5 KiB/s) with 1 file(s) remaining  
Completed 1.5 MiB/7.5 MiB (186.1 KiB/s) with 1 file(s) remaining  
Completed 1.8 MiB/7.5 MiB (195.3 KiB/s) with 1 file(s) remaining  
Completed 2.0 MiB/7.5 MiB (208.7 KiB/s) with 1 file(s) remaining  
Completed 2.2 MiB/7.5 MiB (223.3 KiB/s) with 1 file(s) remaining  
Completed 2.5 MiB/7.5 MiB (222.2 KiB/s) with 1 file(s) remaining  
Completed 2.8 MiB/7.5 MiB (231.0 KiB/s) with 1 file(s) remaining  
Completed 3.0 MiB/7.5 MiB (239.6 KiB/s) with 1 file(s) remaining  
Completed 3.2 MiB/7.5 MiB (229.3 KiB/s) with 1 file(s) remaining  
Completed 3.5 MiB/7.5 MiB (232.0 KiB/s) with 1 file(s) remaining  
Completed 3.8 MiB/7.5 MiB (227.8 KiB/s) with

In [18]:
#https://s3.amazonaws.com/filestogeaux.garyscorner.net/pub/STRMap.html
!aws s3 cp s3://requesterpays.garyscorner.net/datasets/html/STRMap.html s3://filestogeaux.garyscorner.net/pub/ --acl public-read

Completed 7.5 MiB/7.5 MiB (10.1 MiB/s) with 1 file(s) remaining
copy: s3://requesterpays.garyscorner.net/datasets/html/STRMap.html to s3://filestogeaux.garyscorner.net/pub/STRMap.html


In [19]:
print(f"Finished {datetime.now()}")

Finished 2023-07-24 01:21:35.917090
